# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("utils.jl");
include("lattice.jl");
include("basis.jl");
include("sparseHam.jl");
include("sparseS2.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 6;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# NNN coupling
const J2 = 0.4;
# plaquette coupling
const K = 0.3;

# choose Sz sector by specifying number of 1s in basis states
const n1 = convert(Int64,N/2);
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 0;
const my = 0;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;

# number of eigenvalues desired
const numEigs = 6;
# a tolerance for error
const tolerance = 10.^(-5.);
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = true;
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 20;
# maximum number of iterations to converge eigenvalues
const maxIter = 300;

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(n1,kx,ky);

In [5]:
# construct the basis
# 0.35 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = SzkxkyBasis(l,s);
println("Dimension of reduced Hilbert space is ",basis.dim,".");

In [6]:
# using ProfileView;

In [7]:
# Profile.clear();
# @profile SzkxkyBasis(l,s);
# ProfileView.view()

#### build the Hamiltonian

In [8]:
# couplings type to make passing J1,K easier
const c = couplings(J1,J2,K);

# build the sparse Hamiltonian
# takes 2.35 seconds for 6x4 full J1-K Heisenberg model, H contains ~0.14 GB
@time const H = constructSparseHam(basis,c,s,l);

  3.044183 seconds (8.99 M allocations: 683.456 MB, 3.32% gc time)


In [9]:
# using ProfileView;

In [10]:
# Profile.clear();
# @profile constructSparseHam(basis,c,s,l);
# ProfileView.view()

#### find eigenvalues and eigenvectors

In [11]:
# compute eigenvalues
# ~4.18 seconds for 6x4, numEigs=4, which=:SR, full J1-K Heisenberg Hamiltonian
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);

 17.119026 seconds (3.89 M allocations: 209.683 MB, 0.67% gc time)


In [12]:
# print energies
println("Energies are: ",real(eigsResult[1]),".");
# print algorithm performance
println("Number of iterations = ",eigsResult[3]);
println("Number of matrix-vector multiplications = ",eigsResult[4]);

Energies are: [-10.2185,-10.0817,-9.93512,-9.86585,-9.71941,-9.6397].
Number of iterations = 6
Number of matrix-vector multiplications = 33


#### build the $S^2$ operator

In [13]:
# takes 5.05 seconds for 6x4 lattice, S2 contains ~0.3 GB
@time const S2 = constructSparseS2(basis,s,l);

  5.580096 seconds (449.78 k allocations: 443.480 MB, 1.89% gc time)


#### save spectral data
Save in format |  E  |  S(S+1)  |  Sz  |  mx  |  my  |

In [14]:
using DataFrames;

In [15]:
# energies
EData = real(eigsResult[1]);
# Sz values
SzData = fill((N-2*n1)/2,numEigs);
# mx values
mxData = fill(mx,numEigs);
# my values
myData = fill(my,numEigs);

In [17]:
# S(S+1) values
S2Data = zeros(Int64,numEigs);
psi = Array{Complex128,1}(basis.dim);
S2psi = Array{Complex128,1}(basis.dim);
@time for i in 1:numEigs
    psi = eigsResult[2][:,i];
    S2psi = S2*psi;
    S2Data[i] = round(Int64,real(dot(psi,S2psi))[1]);
end;

  0.439745 seconds (60 allocations: 20.656 MB, 2.35% gc time)


In [18]:
# create DataFrame
df = DataFrame(E=EData,Ssqrd=S2Data,Sz=SzData,mx=mxData,my=myData);

In [19]:
println(df)

6×5 DataFrames.DataFrame
│ Row │ E        │ Ssqrd │ Sz  │ mx │ my │
├─────┼──────────┼───────┼─────┼────┼────┤
│ 1   │ -10.2185 │ 0     │ 0.0 │ 0  │ 0  │
│ 2   │ -10.0817 │ 0     │ 0.0 │ 0  │ 0  │
│ 3   │ -9.93512 │ 0     │ 0.0 │ 0  │ 0  │
│ 4   │ -9.86585 │ 0     │ 0.0 │ 0  │ 0  │
│ 5   │ -9.71941 │ 6     │ 0.0 │ 0  │ 0  │
│ 6   │ -9.6397  │ 0     │ 0.0 │ 0  │ 0  │


In [19]:
# dataFileName = "specData/Sz=" * string(SzData[1]) * "_mx=" * string(mxData[1]) * "_my=" * string(myData[1]) * ".csv";

In [20]:
# writetable(dataFileName, df)
# df1 = readtable("specData/Sz=0.0_mx=0_my=0.csv");